# Lab-02-1 linear regression

In [1]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
tf.config.list_physical_devices()

Num GPUs Available:  1


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

> * "tf.set_random_seed(0)" -> "tf.random.set_seed(0)"  
> * https://toramko.tistory.com/entry/tensorflow-%ED%85%90%EC%84%9C%ED%94%8C%EB%A1%9C%EC%9A%B0-setrandomseed-v1-v2-%EC%BD%94%EB%93%9C-%EC%97%85%EA%B7%B8%EB%A0%88%EC%9D%B4%EB%93%9C-1

> tf.set_random_seed(x)  
> 랜덤한 값을 다른 컴퓨터에도 동일하게 얻을 수 있게 해준다.  
> 예를 들어, x = 777 이면 10 14 25 라는 값이 나오고  
> x = 500 이면 20 8 15 라는 값이 나온다.

In [2]:
import tensorflow as tf
tf.random.set_seed(777)

for i in range(10):
    tf.print(tf.random.uniform([1]))

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2022-08-26 16:25:48.295420: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-08-26 16:25:48.295556: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


[0.693055034]
[0.421349049]
[0.909613]
[0.146002769]
[0.0658172369]
[0.135742903]
[0.125757098]
[0.299276471]
[0.957744956]
[0.451489687]


# X and Y data

In [3]:
x_train = [1, 2, 3]
y_train = [1, 2, 3]

# Variable

> "tf.random_normal([1])" -> "tf.random.normal([1])"

In [4]:
# Try to find value for W and b to compute y_data = x_data * W + b
# We know that W should be 1 and b should be 0
# But let's TensorFlow figure it out 

# "tf.Variable()" : trainable variable = tensorflow가 학습하는 과정에서 자기가 변경을 시킨다.
# "tf.random.normal([1])" : shape(rank = 1), 처음에는 W와 b의 값을 모르기 때문에 random한 값을 주게 된다.
W = tf.Variable(tf.random.normal([1]), name='weight')
b = tf.Variable(tf.random.normal([1]), name='bias')

print(W, b)
tf.print(W, b)

<tf.Variable 'weight:0' shape=(1,) dtype=float32, numpy=array([-1.2832785], dtype=float32)> <tf.Variable 'bias:0' shape=(1,) dtype=float32, numpy=array([0.00882021], dtype=float32)>
[-1.28327847] [0.00882021]


# Our Model

<img width="278" alt="스크린샷 2022-07-19 오후 2 30 07" src="https://user-images.githubusercontent.com/91893721/179672331-b64a1bfa-8821-4a83-83b4-035acfeabf64.png">

 <br/>

<img width="497" alt="스크린샷 2022-07-19 오후 2 29 57" src="https://user-images.githubusercontent.com/91893721/179672372-cf911df3-ea7b-41c5-8ace-c1c45428181b.png">


<br/>

> def 위에 "@tf.fucnction" annotation을 붙이면 마치 tf2.x 버전에서도 tf1.x 형태(Session)처럼 그래프 생성과 실행이 분리된 형태로 해당 함수내의 로직이 실행되게 된다.

In [5]:
@tf.function
def cost_eval():
    hypothesis = W * x_train + b
    cost = tf.reduce_mean(tf.square(hypothesis - y_train))
    return cost

In [6]:
# "tf.reduce_mean()"" 예시
t = [1., 2., 3., 4.]
print(tf.reduce_mean(t))
print(tf.reduce_mean(t).numpy())
tf.print(tf.reduce_mean(t))

tf.Tensor(2.5, shape=(), dtype=float32)
2.5
2.5


# Minimize

### Cost를 minimize 하는 작업  
> Cost를 minimize 하는 여러 방법 중 하나 : GradientDescent (경사 하강법)  
<br /> 
> "v1.train.AdamOptimizer"나 "v1.train.GradientDescentOptimizer" 같은 "v1.train"에 있는 옵티마이저는 "tf.keras.optimizers"에 있는 것과 동일하다.  
> "v1.train.GradientDescentOptimizer"는 "tf.keras.optimizers.SGD"로 바꿀 수 있다.  
<br />
> https://www.tensorflow.org/guide/migrate?hl=ko

In [7]:
# 방법 1
optimizer = tf.optimizers.SGD(learning_rate=0.1)

# 방법 2
# Stochastic Gradient Descent
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)

In [8]:
for step in range(2001):
    optimizer.minimize(cost_eval, var_list=[W,b])
    if step % 200 == 0:
        print(step, cost_eval().numpy(), W.numpy(), b.numpy())

2022-08-26 16:25:49.167263: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-08-26 16:25:49.167424: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-08-26 16:25:49.379747: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-08-26 16:25:49.666934: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


0 19.188343 [-1.0705253] [0.09997495]
200 0.03929066 [0.76978123] [0.5233413]
400 0.015003109 [0.85773873] [0.32339323]
600 0.005728931 [0.91209114] [0.19983745]
800 0.0021875869 [0.94567764] [0.12348738]
1000 0.0008353278 [0.9664321] [0.07630777]
1200 0.0003189685 [0.9792571] [0.04715354]
1400 0.0001217972 [0.98718214] [0.02913794]
1600 4.65083e-05 [0.9920793] [0.01800556]
1800 1.7758957e-05 [0.99510556] [0.01112632]
2000 6.7809297e-06 [0.99697554] [0.00687528]


# 코드 정리

In [9]:
x_train = [1, 2, 3]
y_train = [1, 2, 3]


W = tf.Variable(tf.random.normal([1]), name='weight')
b = tf.Variable(tf.random.normal([1]), name='bias')


@tf.function
def cost_eval():
    hypothesis = W * x_train + b
    cost = tf.reduce_mean(tf.square(hypothesis - y_train))
    return cost


# optimizer = tf.optimizers.SGD(learning_rate=0.1)

# Stochastic Gradient Descent
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)


for step in range(2001):
    optimizer.minimize(cost_eval, var_list=[W,b])
    if step % 200 == 0:
        print(step, cost_eval().numpy(), W.numpy(), b.numpy())

0 0.12733594 [0.58950275] [0.69853127]


2022-08-26 16:25:52.395568: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-08-26 16:25:52.407140: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-08-26 16:25:52.425759: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


200 0.029719565 [0.79977554] [0.45515725]
400 0.011348382 [0.8762734] [0.2812595]
600 0.0043333666 [0.92354447] [0.17380133]
800 0.0016546942 [0.95275515] [0.1073987]
1000 0.0006318431 [0.9708056] [0.06636586]
1200 0.00024126662 [0.98195964] [0.04100996]
1400 9.2127375e-05 [0.98885214] [0.02534173]
1600 3.5179073e-05 [0.9931113] [0.01565962]
1800 1.3432746e-05 [0.9957433] [0.00967661]
2000 5.1291354e-06 [0.9973696] [0.0059795]
